In [1]:

import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import cv2
from torch.utils.data import TensorDataset, DataLoader

def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
img_data_2 = np.load('data9.npy')
img_label2 = np.load('label9.npy')


# Apply binary thresholding to all the data
_, train_data = cv2.threshold(img_data_2, 127, 255, cv2.THRESH_BINARY)

# Convert to PyTorch tensors and add channel dimension
train_data_tensor = torch.tensor(train_data, dtype=torch.float32).unsqueeze(1)  # Shape: [num_samples, 1, 40, 168]

train_labels_tensor = torch.tensor(img_label2, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data = train_dataloader

# Load models
resnet_model = torch.load('part2/best_resnet34.pth', map_location=device)



# Define the ResNet-34 model with 1 input channel and correct number of classes
def create_resnet34(num_classes):
    model = models.resnet34(pretrained=True)
    # Modify the first convolutional layer to accept 1 channel instead of 3
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    # Modify the fully connected layer for the correct number of classes
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

# Assuming you have 9 classes (based on 'label9.npy')
num_classes = 37
resnet_model = create_resnet34(num_classes).to(device)

# Load the saved state_dict into the model
resnet_model.load_state_dict(torch.load('part2/best_resnet34.pth', map_location=device))
resnet_model.eval()  # Set to evaluation mode

print("\nEvaluating models on test data...")
resnet_accuracy = evaluate_model(resnet_model, test_data, device)
print(f"ResNet Accuracy: {resnet_accuracy:.2f}%") 

Using device: cuda


/tmp/ipykernel_26088/3828774913.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet_model = torch.load('part2/best_resnet34.pth', map_location=device)
/home/chinmay/


Evaluating models on test data...
ResNet Accuracy: 89.30%
